In [ ]:
# Workflow Agents in Google ADK: Itinerary-Driven Packing List Example
# Welcome 👋🏻 In this notebook, you'll learn how to use Workflow Agents in Google's Agent Development Kit (ADK)
# to chain together two LLM agents: one that creates a travel itinerary, and another that generates a packing list
# based on that itinerary. This mirrors how real travelers plan: first decide what you'll do, then pack accordingly!

<a href="https://colab.research.google.com/github/vinaykrshnn-git2026/google-adk-agentic-exercice/blob/main/03_MultiAgent_Sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

In [ ]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

In [ ]:
import os
from google.colab import userdata # Import userdata to access Colab secrets

# The 'genai' module is often used internally by ADK, but direct configuration might not be needed or available.
# from google import genai

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use.
# 'gemini-2.5-flash' is a fast and efficient model.
MODEL = "gemini-2.5-flash"

# IMPORTANT: Get your Google AI API Key from Colab secrets.
# You can obtain an API key from https://aistudio.google.com/app/apikey
# Store it in Colab secrets under the name 'GOOGLE_API_KEY'.
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# SequentialAgent: A specialized agent that chains together multiple sub-agents in a predefined sequence.
from google.adk.agents import SequentialAgent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

#  InMemorySessionService: A simple, in-memory service for managing conversation sessions.
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# google_search: An ADK tool that provides agents with the ability to perform web searches, enabling them to retrieve up-to-date information.
from google.adk.tools import google_search

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

# IPython.display: Used for rendering rich content like Markdown in Jupyter notebooks.
from IPython.display import Markdown, display

In [ ]:
# Itinerary Agent: Definition
# This agent is designed to create a detailed travel itinerary for a given destination and duration.
# It leverages the `google_search` tool to find up-to-date recommendations for attractions, restaurants, and activities.
# The `output_key="itinerary"` specifies that this agent's primary output will be accessible under the key "itinerary",
# which is crucial for passing this output to subsequent agents in a workflow.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, always include it as a Markdown hyperlink like [Item Name](URL). try to add as many hyperlinks as possible!
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
    output_key="itinerary"  # This agent's output will be stored with the key "itinerary".
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

In [ ]:
# Packing List Agent: Definition
# This agent's primary role is to generate a comprehensive packing list.
# Crucially, its instruction includes a placeholder `{itinerary}`, which tells the agent to
# expect and use the output from a previous step (the itinerary_agent in this workflow).
# The `output_key="packing_list"` ensures that its final output is stored under this key.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL,
    description="Generates a packing list based on itinerary, destination, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        The user will provide their destination, trip duration, and a detailed itinerary (including planned activities, locations, and types of events for each day).
        Your task is to generate a complete, practical packing list tailored to the specific activities and locations in the itinerary.

        Itinerary: {itinerary}

        **Thoroughly analyze the provided Itinerary, considering the specific activities, locations, and time of day for each entry.** Determine necessary clothing, gear, and items based *directly* on these plans. For example, if the itinerary mentions hiking, suggest hiking boots. If it mentions a fancy dinner, suggest formal wear.

        Organize the packing list by category (such as Clothing, Toiletries, Electronics, Documents, and Other Essentials).
        If you notice a special activity or requirement in the itinerary, include specific items for it and justify their inclusion based on the activity.
        If any information is missing, make reasonable assumptions based on the itinerary and destination—do not ask the user for more details unless absolutely necessary.

        **After the main categorized list, provide a dedicated 'Outfit Suggestions for Key Activities' section.** For 2-3 prominent activities from the itinerary, suggest a complete outfit including clothing, shoes, and any relevant accessories, explaining *why* it's suitable for that specific event.

        Present the complete output in a clear, easy-to-read markdown format. For example:

        #### Clothing
        - Comfortable walking shoes (for city tours and museum visits)
        - Lightweight jacket (for potentially cooler evenings in [Destination])
        - Dress for dinner at a nice restaurant (if formal dining is in itinerary)
        - Swimwear (if beach or pool is in itinerary)
        - ...

        #### Toiletries
        - Toothbrush and toothpaste
        - Sunscreen (for outdoor activities)
        - ...

        #### Electronics
        - Phone and charger
        - Plug adapter (if needed for [Destination])
        - ...

        #### Documents
        - Passport
        - Travel insurance (important for any trip abroad)
        - ...

        #### Other Essentials
        - Daypack for city tours
        - Reusable water bottle

        #### Outfit Suggestions for Key Activities
        *   **Day 1 - Morning (Tokyo National Museum):** Comfortable smart casual. E.g., Dark jeans/trousers, a layered top (blouse or stylish t-shirt), and comfortable, stylish walking shoes. A light scarf or jacket for cooler museum interiors.
        *   **Day 2 - Evening (Ninja Restaurant):** Fun, relaxed evening wear. E.g., A casual dress or nice shirt with dark pants. Comfortable shoes for easy movement.
        *   **Day 3 - Afternoon (Sushi Making Class):** Practical and comfortable. E.g., T-shirt, comfortable pants that allow movement, and closed-toe shoes. Avoid overly loose sleeves for hygiene and safety.

        Only ask clarifying questions if the itinerary is extremely vague or unclear. Otherwise, use your best judgment and generate a full, actionable packing list.
    """,
    output_key="packing_list"  # This agent's output will be stored with the key "packing_list".
)

In [ ]:
# Sequential Agent: Definition
# The SequentialAgent allows you to define a workflow by chaining multiple sub-agents together.
# In this setup, the output of the first sub-agent (itinerary_agent) is automatically
# passed as input to the instruction of the next sub-agent (packing_list_agent)
# through the `{itinerary}` placeholder defined in the packing_list_agent's instruction.
wanderwise_agent = SequentialAgent(
    name="wanderwise_agent",
    sub_agents=[itinerary_agent, packing_list_agent],  # Defines the order of execution.
    description="Plans a travel itinerary and prepares a packing list based on activities planned."
)

In [ ]:
# Import necessary modules for type hinting
from typing import Dict

# Modular Function for Agent Interaction (Corrected and Updated for multiple outputs)
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> Dict[str, str]:
    """
    Runs an interaction with an ADK agent (can be a single Agent or SequentialAgent)
    and returns a dictionary of its final text responses.
    Each response is keyed by the agent's 'name' (event.agent_name),
    or a sequential 'response_N' if the agent_name is not available.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        A dictionary where keys are the `agent_name` of the agent that produced the final response.
        Values are the full text responses.
        Returns an empty dictionary if no final response events are found.
    """
    # Use the provided session service if available, otherwise create a new one.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Collect all final responses
    all_final_responses = []
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            full_text = ''.join(part.text for part in event.content.parts if part.text)
            all_final_responses.append(full_text)
    return all_final_responses

print("Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.")

In [ ]:
# Set up the session and user input
APP_NAME = "wanderwise_app"
USER_ID = "default_user"
SESSION_ID = f"{APP_NAME}_default_session"

#  Create a single instance of the session service.
# Passing a shared instance (`common_session_service`) to `run_adk_agent_interaction`
# is good practice, especially if you had a database-backed session service.
# For InMemorySessionService, it ensures all sessions are managed within the same memory scope.
common_session_service = InMemorySessionService()

# Define a list of diverse travel planning test cases for our workflow agent.
# Each dictionary contains a descriptive label, and the user's input.
test_cases = [
    {
        "label": "Adventure & Nature Trip (Banff)",
        "input_text": "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography.",
    },
    {
        "label": "Culture & History Trip (Rome)",
        "input_text": "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine.",
    },
    {
        "label": "Beach & Relaxation Trip (Maldives)",
        "input_text": "I'm visiting the Maldives for 4 days in January for relaxation and some water activities. 25F.",
    },
    {
        "label": "City Break & Shopping (Paris)",
        "input_text": "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!",
    }
]

print("------ [starting workflow agent interactions] ------\n")

# Iterate through each test case and run the travel planning workflow.
# The 'wanderwise_agent' is the SequentialAgent which orchestrates the
# 'itinerary_agent' followed by the 'packing_list_agent'.
# The 'run_adk_agent_interaction' helper function handles session creation and
# collects all final responses (itinerary and packing list) into a list.
for i, case in enumerate(test_cases):
    print(f"\n--- Example {i+1}: {case['label']} ---")
    print(f"User Input: \"{case['input_text']}\"")

    # Invoke the workflow agent using the modular helper function.
    # We pass the shared 'common_session_service' instance.
    workflow_outputs_list = await run_adk_agent_interaction(
        agent=wanderwise_agent,
        user_id="user"+f"_{i+1}",
        session_id="session"+f"_{i+1}",  # Unique session ID for each test case
        input_text=case["input_text"],
        app_name=APP_NAME,
        session_service=common_session_service # Pass the shared service instance
    )

    # Parse and Display Outputs from the list.
    if workflow_outputs_list:
        for j, text_response in enumerate(workflow_outputs_list):
            display(Markdown(text_response))
    else:
        print(f"No outputs received for Example {i+1}.")

    print(f"--- Example {i+1}: Workflow Agent Interaction Complete ---")


print("\n------ [all workflow agent interactions complete] ------")


In [ ]:
# Congratulations 🎉
# You’ve just built and run a SequentialAgent workflow in ADK that first creates an itinerary,
# then generates a packing list tailored to your actual plans.
# Try changing the user input or expanding the workflow with more steps!